In [ ]:
import ibis

In [ ]:
import os

os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""
os.environ["AWS_DEFAULT_REGION"] = "ap-southeast-1"

In [ ]:
user_id = "user_<UPDATE ME>"

In [ ]:
con = ibis.athena.connect(s3_staging_dir="s3://athena-process/temp/")

In [ ]:
table: ibis.Table = con.table("base_table", database="learning-session-db")

In [ ]:
"""
SELECT * LIMIT 10 
FROM "learning-session-db"."base_table";
"""

table.head().execute()

In [ ]:
"""
SELECT id,
	COUNT(value) AS count_value
FROM "learning-session-db"."base_table"
GROUP BY id;
"""

aggregated_table = table.group_by("id").aggregate(
    table.value.count().name("count_value")
)
aggregated_table.execute()

In [ ]:
"""
CREATE TABLE IF NOT EXISTS "learning-session-db"."new_ibis_table_<user_id>" WITH(
	partitioning = ARRAY [ 'bucket(id, 16)' ],
	location = 's3://athena-process/learning-session-db/tables/new_ibis_table_<user_id>',
	table_type = 'ICEBERG',
	is_external = false
) AS
SELECT id,
	COUNT(value) AS count_value
FROM "learning-session-db"."base_table"
GROUP BY id;
"""

con.create_table(
    f"new_ibis_table_{user_id}",
    aggregated_table,
    database="learning-session-db",
    properties={
        "table_type": "ICEBERG",
        "is_external": False,
        "location": f"s3://athena-process/learning-session-db/tables/new_ibis_table_{user_id}",
    },
)


In [ ]:
# Verify the table exists
con.table(f"new_ibis_table_{user_id}", database="learning-session-db").head().execute()